In [ ]:
print('train data shape :', train.shape)
print('test data shape :', test.shape)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  ## IMPORTING LIBRARIES

In [ ]:
# dataframe and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

## LOAD DATASETS FROM KAGGLE

In [ ]:
train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
ss = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')

In [ ]:
train.info()

## EDA

In [ ]:
test.info()

In [ ]:
ss.head()

COMBINE ALL DATA FOR EASY PRE PROCESSING

In [ ]:
# Getting the last index of training data
ntrain = train.shape[0]

# Combining the 2 Dataframes
all_data = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
ntrain

In [ ]:
len(all_data.index) 

In [ ]:
all_data.info()

In [ ]:
all_data.describe()

shrink the size to reduce memory usage

In [ ]:
#reduce the size of the memory usage by the training set
from fastai.tabular.core import df_shrink
all_data= df_shrink(all_data)
print(all_data.memory_usage().sum()/2**20)

In [ ]:
all_data.info()

In [ ]:
pd.set_option('display.max_columns', None) #to display all the columns
all_data.head()


In [ ]:
# Check for missing values
pd.set_option('display.max_rows', None) #display all the rows
print('missing values:', all_data.isnull().sum())

DROP NULL VALUS

In [ ]:
all_data = all_data.dropna()

In [ ]:
print('missing values:', all_data.isnull().sum())

In [ ]:
 #show some information about the dataset
print(all_data.info())
print(all_data.dtypes)


DROP COLUMNS

In [ ]:
column_headers = list(all_data.columns.values)
print("The Column Header :", column_headers)

In [ ]:
all_data = all_data.drop(['climateregions__climateregion'], axis=1)

Extract year, month and date from start data

In [ ]:
all_data['startdate'] = pd.to_datetime(all_data['startdate'])
all_data['year'] = all_data['startdate'].dt.year
all_data['month'] = all_data['startdate'].dt.month
all_data['date'] = all_data['startdate'].dt.day

In [ ]:
print(all_data.dtypes)

In [ ]:
#drop startdate column
all_data = all_data.drop(['startdate'], axis=1) 

In [ ]:
print(all_data.dtypes)

In [ ]:
all_data.info()

In [ ]:
print('missing values:', all_data.isnull().sum())

In [ ]:
all_data = all_data.dropna()

MODEL BUILDING

In [ ]:
all_data.info()

split data into train and test

In [ ]:
train_cleaned = all_data[:ntrain]
test_cleaned = all_data[ntrain:].reset_index(drop=True).drop('contest-tmp2m-14d__tmp2m', axis=1)

In [ ]:
train_cleaned = train_cleaned.dropna()

In [ ]:
print('missing values:', train_cleaned.isnull().sum())

In [ ]:
print('missing values:', test_cleaned.isnull().sum())

In [ ]:
train_cleaned.info()

In [ ]:
test_cleaned.info()

In [ ]:
x_train = train_cleaned.drop(['contest-tmp2m-14d__tmp2m'], axis=1) 
y_train = train_cleaned['contest-tmp2m-14d__tmp2m']

In [ ]:
print('missing values:', y_train.isnull().sum())

In [ ]:
import sklearn.model_selection
# Split train_data
from sklearn.model_selection import train_test_split

X_Train, X_Val, y_Train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.1, random_state=42)

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

In [ ]:
x_train.dtypes

In [ ]:
pip install pycaret

In [ ]:
from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
print('missing values:', y_train.isnull().sum())

In [ ]:
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_Val)
r2_score(y_val, y_pred)

In [ ]:
ss.head()

In [ ]:
test_cleaned.head(5)

In [ ]:
preds = model.predict(test_cleaned)
preds = pd.DataFrame(preds, columns=['contest-tmp2m-14d__tmp2m'])
preds.head()

## SUBMISSION

In [ ]:
index = test_cleaned['index']

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame(
    {
        'contest-tmp2m-14d__tmp2m': preds['contest-tmp2m-14d__tmp2m'],
        'index': index
      
    }
)

submission.head()

In [ ]:
# Create submission csv file csv file
submission.to_csv('submission.csv', index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')